In [1]:
import pandas as pd
from tqdm.notebook import tqdm
import sqlalchemy as sa
from dotenv import load_dotenv
import os

In [2]:
# We have the countries.json -> link -> dataframe using pandas.
# Iterate over the countries dataframe -> legislative_csv_urls
# Moving to a database.-> to_sql method to move it to a database.

In [3]:
df = pd.read_json("https://cdn.rawgit.com/everypolitician/everypolitician-data/080cb46/countries.json")

In [4]:
df.head()

,name,country,code,slug,legislatures
0,Abkhazia,Abkhazia,GE-AB,Abkhazia,"[{'name': 'People's Assembly', 'slug': 'Assemb..."
1,Afghanistan,Afghanistan,AF,Afghanistan,"[{'name': 'Wolesi Jirga', 'slug': 'Wolesi-Jirg..."
2,Albania,Albania,AL,Albania,"[{'name': 'Kuvendi', 'slug': 'Assembly', 'sour..."
3,Alderney,Alderney,GG-ALD,Alderney,"[{'name': 'States', 'slug': 'States', 'sources..."
4,Algeria,Algeria,DZ,Algeria,"[{'name': 'People's National Assembly', 'slug'..."


In [5]:
def get_csv_urls(df):
    csv_dict = {}
    for idx, row in df.iterrows():
        country_name = row["country"]
        country_code = row["code"]
        for leg in row["legislatures"]:
            leg_name = leg["name"]
            for leg_period in leg["legislative_periods"]:
                st_dt = leg_period["start_date"]
                if "end_date" in leg_period.keys():
                    ed_dt = leg_period["end_date"]
                else:
                    ed_dt = ""
                csv_url = leg_period["csv_url"]
                csv_dict[(country_name, country_code, leg_name, st_dt, ed_dt)] = csv_url
    return csv_dict

In [6]:
urls = get_csv_urls(df)

In [7]:
def get_politician_data(pol_urls):
    all_politician_df = []
    for keys, value in tqdm(pol_urls.items()):
        try:
            politician_df = pd.read_csv(value)
            politician_df["country_name"] = keys[0]
            politician_df["country_code"] = keys[1]
            politician_df["leg_name"] = keys[2]
            politician_df["start_date_legislature"] = keys[3]
            politician_df["end_date_legislature"] = keys[4]
            all_politician_df.append(politician_df)
        except Exception as e:
            print(f"For the url {value} got error {e}")
    return pd.concat(all_politician_df)
        
        

In [8]:
pol_df = get_politician_data(urls)

For the url https://cdn.rawgit.com/everypolitician/everypolitician-data/0b326e2/data/Ecuador/Asamblea/term-2013.csv got error HTTP Error 404: Not Found
For the url https://cdn.rawgit.com/everypolitician/everypolitician-data/8102167/data/Montserrat/Assembly/term-1.csv got error HTTP Error 404: Not Found
For the url https://cdn.rawgit.com/everypolitician/everypolitician-data/fbd3ca5/data/Norway/Storting/term-1969-73.csv got error HTTP Error 404: Not Found
For the url https://cdn.rawgit.com/everypolitician/everypolitician-data/fbd3ca5/data/Norway/Storting/term-1965-69.csv got error HTTP Error 404: Not Found
For the url https://cdn.rawgit.com/everypolitician/everypolitician-data/6baa026/data/Seychelles/Assembly/term-2011.csv got error HTTP Error 404: Not Found
For the url https://cdn.rawgit.com/everypolitician/everypolitician-data/fedacb2/data/Turkmenistan/Mejlis/term-5.csv got error HTTP Error 404: Not Found
For the url https://cdn.rawgit.com/everypolitician/everypolitician-data/fedacb2/d

In [9]:
usa = pol_df[pol_df.country_code=='US']

In [9]:
pol_df.to_csv("politician_data.csv", index=False)

In [10]:
pol_df = pd.read_csv("politician_data.csv", encoding='utf-8')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [12]:
from sqlalchemy import create_engine

load_dotenv()

connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
    os.getenv("user"),
    os.getenv("password"),
    os.getenv("host"),
    os.getenv("database")
)


In [13]:
def to_alchemy(df):

    engine = create_engine(connect)
    df.to_sql(
        'usa_politician', 
        con=engine, 
        index=False, 
        if_exists='replace'
    )
    print("to_sql() done (sqlalchemy)")

In [14]:
to_alchemy(usa)

to_sql() done (sqlalchemy)


In [15]:
# Figure, out the unicode for entering the other characters to a database.

In [14]:
from sqlalchemy import create_engine

load_dotenv()

connect = "postgresql+psycopg2://%s:%s@%s:5432/%s" % (
    os.getenv("user"),
    os.getenv("password"),
    os.getenv("host"),
    os.getenv("database")
)


In [15]:
def to_alchemy(df):

    engine = create_engine(connect)
    df.to_sql(
        'every_politician', 
        con=engine, 
        index=False, 
        if_exists='replace'
    )
    print("to_sql() done (sqlalchemy)")

In [16]:
to_alchemy(pol_df)

to_sql() done (sqlalchemy)
